## Classification des critiques cinématographiques

### 1. Les données

#### 1.1 Téléchargement

In [19]:
import pandas as pd

# Le fichier labeldedTrainData.tsv contient les données labélisées que nous
# utiliser pour l'apprentissage. 
train = pd.read_csv("labeledTrainData.tsv", header=0,delimiter="\t", quoting=3)

# On affiche la taille du fichier de données
print 'Size of labeledTrainData.tsv: ', train.shape
print ''

# Le fichier labeledTrainData.tsv est composé de trois colonnes que l'on affiche
print 'Values on labeledTrainData.tsv: ', train.columns.values
print''

# Par exemple, on peut afficher la review et le sentiment du 10ième film
# 0 : bad
# 1 : good
print 'Sentiment: ', train["sentiment"][10]
print 'Review:', train["review"][10]
print ''

Size of labeledTrainData.tsv:  (25000, 3)

Values on labeledTrainData.tsv:  ['id' 'sentiment' 'review']

Sentiment:  0
Review: "What happens when an army of wetbacks, towelheads, and Godless Eastern European commies gather their forces south of the border? Gary Busey kicks their butts, of course. Another laughable example of Reagan-era cultural fallout, Bulletproof wastes a decent supporting cast headed by L Q Jones and Thalmus Rasulala."



#### 1.2 Nettoyage

Avant de pouvoir utiliser les reviews, nous devons les nettoyer i.e. ne conserver que les lettres de l'alphabet, en minuscule. Ceci est possible grâce à une bibliothèque python traitant les [expressions régulières](https://docs.python.org/2/library/re.html#).

Dans certains avis, il y a des balises HTML. Pour les enlever, on va utiliser la librairie [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [22]:
# On importe la bibliothèque re pour traiter les expressions régulières.
import re
# On importe Beautiful Soup.
import bs4 as bs

# Par exemple, la review du premier film contient des caractères HTML
print train["review"][0]
print''

# On peut les supprimer de cette façon : 
example1 = bs.BeautifulSoup(train["review"][0])
print example1.get_text()
print''

# Maintenant, on ne conserve que les lettres de l'alphabet.
letters_only = re.sub("[^a-zA-Z]"," ",example1.get_text())
print letters_only
print''

# On les met en minuscule.
lower_case = letters_only.lower()
print lower_case
print ''

# Enfin, on sépare la revue en mots.
words = lower_case.split()
print words

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

La dernière opération consiste à enlever les mots vide de sens (stop words en anglais). Pour faire ça, on va utiliser la bibliothèque python [Natural Language Toolkit](http://www.nltk.org/).

In [23]:
# On importe la bibliothèque nltk
import nltk

# L'étape suivante permet de télécharger tout ce dont nous avons besoin.
# Je n'ai pas pu la faire pour des problèmes d'autorization du proxy.
# J'ai donc télécharger manuellement la bibliothèque pour les stop words via
# http://www.nltk.org/nltk_data/

# nltk.set_proxy('http://ba-proxy.sesame.infotel.com:8080',
#               ('m_bess', '1nf0TL@EPO'))
# nltk.download()

In [26]:
# On import la package stopwords de nltk.corpus
from nltk.corpus import stopwords

# On peut visualiser les stop words de plusieurs langues comme le français 
# ou l'anglais.
# print stopwords.words("french")
# print''
# print stopwords.words("english")
# D'autres langues sont également disponibles (voir C:\Users\m_bess\
#                                                      AppData\Roaming\
#                                                      nlyk_data\corpus\stopwords).

# Dans notre exemple ci-dessus, words, on va supprimer les stop words.
words = [w for w in words if not w in stopwords.words("english")]
print words

[u'stuff', u'going', u'moment', u'mj', u've', u'started', u'listening', u'music', u'watching', u'odd', u'documentary', u'watched', u'wiz', u'watched', u'moonwalker', u'maybe', u'want', u'get', u'certain', u'insight', u'guy', u'thought', u'really', u'cool', u'eighties', u'maybe', u'make', u'mind', u'whether', u'guilty', u'innocent', u'moonwalker', u'part', u'biography', u'part', u'feature', u'film', u'remember', u'going', u'see', u'cinema', u'originally', u'released', u'subtle', u'messages', u'mj', u'feeling', u'towards', u'press', u'also', u'obvious', u'message', u'drugs', u'bad', u'm', u'kay', u'visually', u'impressive', u'course', u'michael', u'jackson', u'unless', u'remotely', u'like', u'mj', u'anyway', u'going', u'hate', u'find', u'boring', u'may', u'call', u'mj', u'egotist', u'consenting', u'making', u'movie', u'mj', u'fans', u'would', u'say', u'made', u'fans', u'true', u'really', u'nice', u'actual', u'feature', u'film', u'bit', u'finally', u'starts', u'minutes', u'excluding', u's

In [29]:
# On peut résumer toutes les étapes de nettoyage dans une seule fonction.
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. On supprime les balises HTML.
    review_text = bs.BeautifulSoup(raw_review).get_text() 
    
    # 2. On supprime tout ce qui n'est pas une lettre.        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 

    # 3. On met les lettres en minuscule.
    words = letters_only.lower().split()                             

    # 4. On supprime les stop words de la liste de mots.
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in set(stopwords.words("english"))]   

    # 5. On retourne une chaîne de mots séparés par des espaces.
    return( " ".join( meaningful_words )) 

In [30]:
# Exemple d'utilisation de la fonction review_to_words avec la review du
# premier film.
#clean_review = review_to_words( train["review"][0] )
#print clean_review

In [42]:
clean_train_reviews = [review_to_words(train["review"][review])
 for review in range(train["review"].size)]

Maintenant que les données sont nettoyées correctement, il faut trouver un mieyn de les convertire en données numériques pour pouvoir faire dur machine learning (ML) à proprement parlé.

#### 1.3 Mise en forme numérique des données

Une moyen de transformer les données en quelque chose de numérique est d'utiliser l'approche [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model). 

La méthode commence par apprendre du vocabulaire en analysant tous les documents. Ensuite, elle compte l'occurrence de chaque mots contenu dans chaque documents.

Il est possible de créer un Bag of Words en utilisant le module [feature_extraction](http://scikit-learn.org/stable/modules/feature_extraction.html) de [scikit-learn](http://scikit-learn.org/stable/).

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

# On initialise un CountVectorizer.  
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 3000) 
# Ici on ne conserve que 5000 mots pour le vocabulaire, c'est largement 
# suffisant.

# L'apprentissage du vocabulaire se fait avec la fonction fit_transform.
train_data_features = vectorizer.fit_transform(clean_train_reviews).toarray()
print 'Size of train_data_features: ',train_data_features.shape

Size of train_data_features:  (25000L, 3000L)


In [ ]:
# On peut avoir accès au vocabulaire généré de la façon suivante : 
vocab = vectorizer.get_feature_names()
print vocab

Maintenant que nous avons des données numériques, on peut passer à la partie ML.

### 2. Partie Machine Learning

Nous possèdons maintenant un couple de variables $(x,y)$ avec $x$ les variables d'entrée et $y$ les variables de réponse.

La première étape consiste à choisir l'algorithme que nous allons utiliser. Ici, nous devons faire de la classification, on peut donc utilises des SVM, des forêts aléatoires, l'algorithme des $k$ plus proches voisins etc.

Une fois que la méthode est sélectionnée, on entraîne le modèle avec notre couple de variables. Ici, le rôle de $x$ est joué par train_data_features et celui de $y$ par le label sentiment.

Quand le modèle est entraîné, on peut se lancer dans le sprédictions.

#### 2.1 Forêt aléatoires

In [57]:
from sklearn.ensemble import RandomForestClassifier

# On initialise la forêt aléatoire avec 100 arbres
forest = RandomForestClassifier(n_estimators = 100) 

# On lance la partie apprentissage avec le bag of words comme variable d'entrée
# et le label sentiment comme variable de réponse.
forest = forest.fit(train_data_features, train["sentiment"])

In [78]:
# Read the test data
test = pd.read_csv("my_reviews.tsv", header=0, delimiter="\t", \
                   quoting=3 )

clean_test_reviews = [review_to_words(test["review"][review])
                      for review in range(test["review"].size)]
    
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews).toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} ).to_csv( "my_predictions.csv", index=False, quoting=3 )


#### 2.2 Support Vector Machine

In [80]:
from sklearn import svm

clf = svm.SVC()

clf.fit(train_data_features, train["sentiment"])

clf.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} ).to_csv( "my_predictions_SVM.csv", index=False, quoting=3 )

NameError: name 'SVC' is not defined

In [81]:
clf.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} ).to_csv( "my_predictions_SVM.csv", index=False, quoting=3 )